In [1]:
from BcomMEG import *
from TripletMEGDataset import *
from MEGDataset_Conv import *
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader, Subset
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
import torch.nn as nn
class BigTripletNet(nn.Module): #this one is designed to take in all the sensors as one big tensor
    def __init__(self):
        super(BigTripletNet, self).__init__()
        
        #Conv Blocks
        self.conv1 = nn.Conv2d(in_channels=247, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) #(16, 24, 40)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1) 
        self.bn2 = nn.BatchNorm2d(32)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) #(32, 12, 20)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.act3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) #64, 6, 10

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.act4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2) #128, 3, 5

        #FC layers
        self.fc1 = nn.Linear(1920, 512)
        self.fc2 = nn.Linear(512, 128)


    def forward(self, x):
        x = self.pool1(self.act1(self.bn1(self.conv1(x))))
        x = self.pool2(self.act2(self.bn2(self.conv2(x))))
        x = self.pool3(self.act3(self.bn3(self.conv3(x))))
        x = self.pool4(self.act4(self.bn4(self.conv4(x))))


        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x
    
model = BigTripletNet()

In [3]:
from torch.utils.data import Dataset
import random
import numpy as np
import json
import os

class TripletMEGDataset(Dataset):
    def __init__(self, BcomMEG_object, label_map, indices, normalize=False):
        self.data_dict = BcomMEG_object 
        self.label_map = self.get_label_map(label_map)
        self.labels = self.get_labels(self.data_dict, self.label_map)

        self.data = self.data_dict.data_to_tensor(normalize)

        #ONLY TRAINING SAMPLES
        self.indices = indices
        self.data = self.data[self.indices]
        self.labels = [self.labels[i] for i in self.indices]

       # Build an index mapping from labels to indices for efficient sampling
        self.label_to_indices = {}
        for idx, label in enumerate(self.labels):
            if label not in self.label_to_indices:
                self.label_to_indices[label] = []
            self.label_to_indices[label].append(idx)

    def get_label_map(self, map_name)->dict:
        
        '''This method is used by the class to populate its label_map attribute. 
        If a dictionary is passed as the map_name, then this method returns that dictionary.
        If a string is passed, then this method searches the directory for the label_maps.json, and recovers the corresponding
        label map if it is found in the file'''

        if type(map_name) == dict:
            label_map = map_name
        else:
            dir = os.getcwd()
            with open(os.path.join(dir, 'label_maps.json'), 'r') as f:
                label_maps = json.load(f)
                if map_name not in label_maps:
                    legal_names = list(label_maps.keys())
                    raise ValueError(f"map_name '{map_name}' not found in label_maps.json. Legal names are: {legal_names}")
                label_map = label_maps.get(map_name)
        
        return label_map
    

    def get_labels(self, data_dict, label_map) -> list:
        '''This method is used by the class to populate its labels attribute.
        It returns a list which matches the data at each index to its label'''
        labels = []
        syllable_counts = data_dict.get_syllable_counts()
        for subject in syllable_counts:
            for syllable in syllable_counts[subject]:
                labels.extend([label_map[syllable]] * syllable_counts[subject][syllable])
        return labels
    



    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        anchor = self.data[idx]
        anchor_label = self.labels[idx]
        
        # Positive samples from the same class, excluding the current idx
        positive_indices = self.label_to_indices[anchor_label]
        positive_indices = [i for i in positive_indices if i != idx]
        random_positive_index = random.choice(positive_indices)
        positive = self.data[random_positive_index]

        # Negative samples from different classes
        negative_label = random.choice([label for label in self.label_to_indices if label != anchor_label])
        negative_index = random.choice(self.label_to_indices[negative_label])
        negative = self.data[negative_index]

        return anchor, positive, negative

In [4]:
dir = '/Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT'
subject = ['BCOM_18_2', 'BCOM_18_3', 'BCOM_18_4']
avoid_reading=True
picks=None
frequencies = np.array(range(2, 100, 2))
divisor = 7
baseline = (None, 0)

data = BcomMEG(subjects=subject, 
               dir=dir, 
               picks=picks, 
               avoid_reading=True)

Reading /Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT/BCOM_18_2_re_144-epo.fif ...
    Found the data of interest:
        t =    -300.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
8 matching events found
No baseline correction applied
0 projection items activated
Reading /Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT/BCOM_18_2_i_116-epo.fif ...
    Found the data of interest:
        t =    -300.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
12 matching events found
No baseline correction applied
0 projection items activated
Reading /Volumes/@neurospeech/PROJECTS/BCI/BCOM/DATA_ANALYZED/EVOKED/DATA/WITHOUT_BADS/COVERT/BCOM_18_2_a_112-epo.fif ...
    Found the data of interest:
        t =    -300.00 ...     500.00 ms
        0 CTF compensation matrices available
Not setting metadata
16 matching events found
No baseline correct

In [5]:
transformed_data = data.get_spectrogram(frequencies=frequencies, 
                            baseline=baseline, 
                            cycle_divisor=divisor,
                            mode='zscore',
                            data_only=True)

NB! Data_only is set to true. To get accurate plots, the extent will have to be set manually, and channels 
will have to be specified as indexes, not names
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


Applying baseline correction (mode: zscore)


[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    1.9s
[Parallel(n_jobs=10)]: Done 172 tasks      | elapsed:    2.1s
[Parallel(n_jobs=10)]: Done 247 out of 247 | elapsed:    2.1s finished


NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").
Applying baseline correction (mode: zscore)
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


...

[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 172 tasks      | elapsed:    0.1s
[Parallel(n_jobs=10)]: Done 247 out of 247 | elapsed:    0.1s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 172 tasks      | elapsed:    0.0s


Applying baseline correction (mode: zscore)
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").
Applying baseline correction (mode: zscore)
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").


[Parallel(n_jobs=10)]: Done 247 out of 247 | elapsed:    0.1s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 172 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 247 out of 247 | elapsed:    0.1s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 172 tasks      | elapsed:    0.0s


Applying baseline correction (mode: zscore)
NOTE: tfr_morlet() is a legacy function. New code should use .compute_tfr(method="morlet").
Applying baseline correction (mode: zscore)


[Parallel(n_jobs=10)]: Done 247 out of 247 | elapsed:    0.1s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 172 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 247 out of 247 | elapsed:    0.1s finished


In [6]:
dataset = MEGDataset_Conv(transformed_data, label_map='covert_vowel_class_label_map')
train_indices, test_indices = train_test_split(
    list(range(len(dataset))), 
    test_size=0.2, 
    random_state=42, 
    stratify=dataset.labels)

train_dataset = TripletMEGDataset(transformed_data, label_map='covert_vowel_class_label_map', indices=train_indices)
test_dataset = Subset((MEGDataset_Conv(transformed_data, label_map='covert_vowel_class_label_map', normalize=False)), test_indices)

# Check for common samples
common_samples = set(train_indices).intersection(test_indices)
if common_samples:
    print(f"Common samples found: {common_samples}")
else:
    print("No common samples found")


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
for i, (anchor, positive, negative) in enumerate(train_loader):
    print(f"Batch {i}")
    print("Anchor:", anchor)
    print("Positive:", positive)
    print("Negative:", negative)
    break

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
for i, (input,label) in enumerate(test_loader):
    print(f"Batch: {i}")
    print(f"Input: {input}")
    print(f"Label: {label}")
    break

No common samples found
Batch 0
Anchor: tensor([[[[-8.5140e-01, -2.2878e-01,  4.8449e-01,  ..., -8.5692e-02,
           -7.1387e-01, -1.0041e+00],
          [-8.4413e-01, -2.4459e-01,  4.3447e-01,  ..., -5.0547e-02,
           -7.1044e-01, -1.0074e+00],
          [-7.9951e-01, -2.5715e-01,  3.4266e-01,  ...,  1.1162e-02,
           -6.7006e-01, -9.6929e-01],
          ...,
          [-1.0168e+00, -5.6361e-01,  6.1673e-02,  ..., -7.9661e-01,
           -1.0529e+00, -1.1247e+00],
          [-7.7905e-01, -3.5856e-01,  1.6629e-01,  ..., -6.4602e-01,
           -8.1622e-01, -9.0747e-01],
          [-6.4334e-01, -2.3133e-01,  2.0910e-01,  ..., -6.5135e-01,
           -7.3900e-01, -8.2175e-01]],

         [[-5.1493e-01,  1.8743e-01,  1.0265e+00,  ...,  7.7813e-01,
           -6.3583e-01, -6.2914e-01],
          [-5.4564e-01,  1.2404e-01,  9.2939e-01,  ...,  9.3605e-01,
           -4.8128e-01, -5.4549e-01],
          [-5.9303e-01,  1.7465e-02,  7.5586e-01,  ...,  1.1792e+00,
           -2.3508

In [7]:
from sklearn.preprocessing import normalize
torch.manual_seed(0)
device = 'mps'
model = model.to(device=device)
criterion = nn.TripletMarginLoss(margin=10.0, p=2)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
epochs = 100
total_steps = len(train_loader)

loss_values = []
model.train()
for epoch in range(epochs):
    for i, (anchor, positive, negative) in enumerate(train_loader):
    
        anchor, positive, negative = anchor.to(device, dtype=torch.float32),positive.to(device, dtype=torch.float32), negative.to(device, dtype=torch.float32)
        
        optimizer.zero_grad()
        anchor_embedding = model(anchor)
        positive_embedding = model(positive)
        negative_embedding = model(negative)

        loss = criterion(anchor_embedding, positive_embedding, negative_embedding)

        
        loss.backward()
        optimizer.step()
        

        loss_values.append(loss.item())

        if (i+1) % 10 == 0:  # Adjusted the condition to a smaller number
            print(f'Epoch [{epoch + 1} / {epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.6f}')
print('Training Finished')


Epoch [1 / 100], Step [10/16], Loss: 10.475430
Epoch [2 / 100], Step [10/16], Loss: 10.355314
Epoch [3 / 100], Step [10/16], Loss: 9.674778
Epoch [4 / 100], Step [10/16], Loss: 9.996514
Epoch [5 / 100], Step [10/16], Loss: 9.968903
Epoch [6 / 100], Step [10/16], Loss: 10.174206
Epoch [7 / 100], Step [10/16], Loss: 9.689634
Epoch [8 / 100], Step [10/16], Loss: 9.889215
Epoch [9 / 100], Step [10/16], Loss: 9.072967
Epoch [10 / 100], Step [10/16], Loss: 10.327597
Epoch [11 / 100], Step [10/16], Loss: 10.125992
Epoch [12 / 100], Step [10/16], Loss: 9.452953
Epoch [13 / 100], Step [10/16], Loss: 9.664639
Epoch [14 / 100], Step [10/16], Loss: 8.198694
Epoch [15 / 100], Step [10/16], Loss: 10.705570
Epoch [16 / 100], Step [10/16], Loss: 10.555359
Epoch [17 / 100], Step [10/16], Loss: 10.060613
Epoch [18 / 100], Step [10/16], Loss: 7.174542
Epoch [19 / 100], Step [10/16], Loss: 7.890179
Epoch [20 / 100], Step [10/16], Loss: 4.303816
Epoch [21 / 100], Step [10/16], Loss: 5.759506
Epoch [22 / 10

In [8]:
#Compute embeddings for KNN
train_clustering = Subset(MEGDataset_Conv(transformed_data, label_map='covert_vowel_class_label_map'), train_indices)
clustering_loader = DataLoader(train_clustering, batch_size=1)

clustering_embeddings = []
clustering_labels = []

model.eval()

with torch.no_grad():
    for i, (inputs, label) in enumerate(clustering_loader):
        inputs = inputs.to(device, dtype=torch.float32)
        embeddings = model(inputs)
        clustering_embeddings.append(embeddings.cpu().numpy())
        clustering_labels.append(label.cpu().numpy())

clustering_embeddings = normalize(np.concatenate(clustering_embeddings, axis=0))
clustering_labels = np.concatenate(clustering_labels, axis=0)


test_embeddings = []
test_labels = []
with torch.no_grad():
    true_labels = []
    for i, (inputs, label) in enumerate(test_loader):
        inputs = inputs.to(device, dtype=torch.float32)
        embeddings = model(inputs)
        test_embeddings.append(embeddings.cpu().numpy())
        test_labels.append(label.cpu().numpy())

test_embeddings = normalize(np.concatenate(test_embeddings, axis=0))
test_labels = np.concatenate(test_labels, axis=0)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
k = 7
knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')  # Euclidean distance in embedding space

# Convert lists to numpy arrays
clustering_embeddings = np.array(clustering_embeddings)
clustering_labels = np.array(clustering_labels)

# Train KNN on training embeddings
knn.fit(clustering_embeddings, clustering_labels)

# Convert test embeddings to numpy array
test_embeddings = np.array(test_embeddings)

# Predict labels for test embeddings
predicted_labels = knn.predict(test_embeddings)

# Evaluate accuracy
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels, average='weighted')

print(f"KNN Accuracy: {accuracy * 100:.2f}%")
print(f"KNN F1 Score: {f1 * 100:.2f}")

KNN Accuracy: 41.32%
KNN F1 Score: 41.38


In [1]:
# Combine clustering and test embeddings for visualization
from sklearn.manifold import TSNE

# Example data: Replace with your embeddings and labels
# clustering_embeddings, test_embeddings, clustering_labels, test_labels
# are assumed to be defined.

# Combine embeddings and labels
all_embeddings = np.concatenate((clustering_embeddings, test_embeddings), axis=0)
all_labels = np.concatenate((clustering_labels, test_labels), axis=0)

# Fit t-SNE
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(all_embeddings)

# Split t-SNE results for clustering and test embeddings
clustering_tsne = tsne_results[:len(clustering_embeddings)]
test_tsne = tsne_results[len(clustering_embeddings):]

# Plot the results
plt.figure(figsize=(10, 8))

# Plot clustering embeddings
scatter_clustering = plt.scatter(
    clustering_tsne[:, 0], clustering_tsne[:, 1],
    c=clustering_labels, cmap='viridis', alpha=0.6, marker='o', label='Clustering Embeddings'
)

# Plot test embeddings
scatter_test = plt.scatter(
    test_tsne[:, 0], test_tsne[:, 1],
    c=test_labels, cmap='viridis', alpha=0.6, marker='D', label='Test Embeddings'
)

# Add legend and colorbar
plt.colorbar(scatter_clustering, label='Labels')
plt.legend(loc='best')
plt.title('t-SNE visualization of embeddings')
plt.xlabel('t-SNE component 1')
plt.ylabel('t-SNE component 2')
plt.show()

NameError: name 'np' is not defined

In [11]:
map = {"a_112": 0, "e_114": 1, "i_116": 2, "la_122": 3, "le_124": 4, "li_126": 5, "ma_132": 6, "me_134": 7, "mi_136": 8, "ra_142": 9, "re_144": 10, "ri_146": 11, "sa_152": 12, "se_154": 13, "si_156": 14, "ta_162": 15, "te_164": 16, "ti_166": 17}

In [12]:
print(map)

{'a_112': 0, 'e_114': 1, 'i_116': 2, 'la_122': 3, 'le_124': 4, 'li_126': 5, 'ma_132': 6, 'me_134': 7, 'mi_136': 8, 'ra_142': 9, 're_144': 10, 'ri_146': 11, 'sa_152': 12, 'se_154': 13, 'si_156': 14, 'ta_162': 15, 'te_164': 16, 'ti_166': 17}
